# K Means Clustering and Gaussian Mixture Models

## Visualizing the data

![](plots/q2_data_viz.png)

## K Means Clustering

I used the ```scipy.cluster.vq.kmeans2``` class to perform the clustering on the dataset.

Since, kmeans2 only returns the clusters and a map to the initial labels is missing, I made an assumption that the left most cluster is ```Ear_left```, the right most cluster is ```Ear_right``` and the last one is the ```Head```.

Below are the confusion matrix and the clustering plots

![](plots/q2_kmeans_output.png)

## Gaussian Mixture Models

Outputs of the Gaussian Mixure Models in the first four iterations

![](plots/q2_gmm_0_iter.png)
![](plots/q2_gmm_1_iter.png)
![](plots/q2_gmm_2_iter.png)
![](plots/q2_gmm_3_iter.png)

Output after satisfying the convergence criteria (Difference between likelihood < $10^-5$)

![](plots/q2_gmm_24_iter.png)

Maximization of the likelihood function

![](plots/q2_likelihood.png)

## Difference between K-Means and Gaussian Mixture Models

* The most important difference between K-Means and Gaussian Mixture Models is hard clustering vs soft clustering
    * K-Means algorithm gives a binary output. A point either belongs to a cluster or it does not
    * Gaussian Mixture Models give a probability that a point could belong to a cluster. The sum of the probabilities for a point across all clusters should be 1
    
* K-Means does not account for variance. K-Means clusters form circles unlike Gaussisn PDFs that can account for very elliptical clusters

**Conclusion**: Gaussian Mixture Models produce better clusters as compared to K-Means clustering

# Python Code Appendix

In [ ]:
import sys
import re
import matplotlib.pyplot as plt
from scipy.cluster.vq import kmeans2
import numpy as np
from scipy.stats import multivariate_normal
from matplotlib import cm

In [ ]:
def load_data(file_name):
    with open(file_name, "r") as file:
        data = file.read().splitlines()
    
    # Define a regex pattern to filter out unnecessary lines from the CSV file
    pattern = "^\d+\.\d+ \d+\.\d+ [a-zA-Z]+_?[a-zA-Z]*"
    data = [i for i in data if re.search(pattern, i)]

    head_coordinates = np.array([[float(j) for j  in i.split(" ")[0:2]] for i in data if i.split(" ")[2] == "Head"])
    ear_left_coordinates = np.array([[float(j) for j  in i.split(" ")[0:2]] for i in data if i.split(" ")[2] == "Ear_left"])
    ear_right_coordinates = np.array([[float(j) for j  in i.split(" ")[0:2]] for i in data if i.split(" ")[2] == "Ear_right"])
    
    return head_coordinates, ear_left_coordinates, ear_right_coordinates, data

# K-Means Clustering

In [ ]:
def generate_confusion_matrix(actual, predicted):
    num_labels = len(np.unique(actual))
    matrix = np.zeros([num_labels, num_labels])
    
    for i in range(len(actual)):
        matrix[actual[i], predicted[i]] += 1
    
    return matrix

In [ ]:
# Define lists to store coordinates of head, ears

head_coordinates, ear_left_coordinates, ear_right_coordinates, data = load_data("mickey.csv")
fig = plt.figure()

plt.scatter(head_coordinates[:,0], head_coordinates[:,1] , color="Blue")
plt.scatter(ear_left_coordinates[:,0], ear_left_coordinates[:,1] , color="Red")
plt.scatter(ear_right_coordinates[:,0], ear_right_coordinates[:,1], color="Green")
plt.axis("equal")
plt.savefig("plots/q2_data_viz.png", dpi=200)
plt.show()

In [ ]:
# Concatenate all the coordinated into one list to feed to scipy kmeans algorithm

train_coordinates = np.array([*head_coordinates, *ear_left_coordinates, *ear_right_coordinates])

centroid, predicted = kmeans2(train_coordinates, 3, minit="points")

In [ ]:
# Assigning labels to the predicted classes. Sort the centroids by the left coordinate and assign Ear_left, Head, Ear_right respectively

labels_sorted = np.argsort(centroid[:,0])
labels_original = ["Ear_left", "Head", "Ear_right"]
labels_dict = dict(zip(labels_original, labels_sorted))
print(labels_dict)
actual = np.array([labels_dict[i.split(" ")[2]] for i in data])

In [ ]:
c_matrix = generate_confusion_matrix(actual, predicted)

In [ ]:
fig, ax = plt.subplots(1,2)
fig.set_size_inches(13,6.5)
ear_left_predicted = train_coordinates[predicted == labels_dict["Ear_left"]]
head_predicted = train_coordinates[predicted == labels_dict["Head"]]
ear_right_predicted = train_coordinates[predicted == labels_dict["Ear_right"]]

fig.axes[0].scatter(head_predicted[:, 0], head_predicted[:, 1], color="Blue")
fig.axes[0].scatter(ear_left_predicted[:, 0], ear_left_predicted[:, 1], color="Red")
fig.axes[0].scatter(ear_right_predicted[:, 0], ear_right_predicted[:, 1], color="Green")
fig.axes[0].scatter(centroid[:, 0], centroid[:, 1], color="Black")
fig.axes[0].axis("equal")
fig.axes[0]

fig.axes[1].matshow(c_matrix, cmap="gray")
fig.axes[1].set_xticks(ticks=list(labels_dict.values()))
fig.axes[1].set_yticks(ticks=list(labels_dict.values()))
fig.axes[1].set_xticklabels(labels=list(labels_dict.keys()), rotation=45)
fig.axes[1].set_yticklabels(labels=list(labels_dict.keys()), rotation=45)
fig.axes[1].set_xlabel("Predicted")
fig.axes[1].set_ylabel("Actual")
for i in range(len(c_matrix)):
    for j in range(len(c_matrix)):
        fig.axes[1].text(j, i, c_matrix[i,j], weight=1000, rotation=45, color="blue")
fig.savefig("plots/q2_kmeans_output.png", dpi=200)
plt.show()

# Gaussian Mixture Models

In [ ]:
NUMBER_OF_CLUSTERS = 3
NUMBER_OF_ITERATIONS = 20
DIMENSIONS = 2

head_coordinates, ear_left_coordinates, ear_right_coordinates, data = load_data("mickey.csv")

# Concatenate all the coordinated into one list to feed to scipy kmeans algorithm
train_coordinates = np.array([*head_coordinates, *ear_left_coordinates, *ear_right_coordinates])

NUM_DATA_POINTS = train_coordinates.shape[0]

In [ ]:
class Clusters:
    
    def __init__(self, NUMBER_OF_CLUSTERS, NUM_DATA_POINTS, DIMENSIONS):
        self.NUM_CLUSTERS = NUMBER_OF_CLUSTERS
        self.NUM_DATA_POINTS = NUM_DATA_POINTS
        self.DIMENSIONS = DIMENSIONS
        self.centroids = np.zeros([NUMBER_OF_CLUSTERS, DIMENSIONS])
        self.covariances = np.zeros([NUMBER_OF_CLUSTERS, DIMENSIONS, DIMENSIONS])
        self.mixing_weights = np.zeros([NUMBER_OF_CLUSTERS, 1])
        self.cluster_probability = np.zeros([NUM_DATA_POINTS, NUMBER_OF_CLUSTERS])
    
    def __str__(self):
        return f"There are {self.NUM_CLUSTERS} with centroids at \n {self.centroids}"
    
# Initialise the clusters
clusters = Clusters(NUMBER_OF_CLUSTERS, NUM_DATA_POINTS, DIMENSIONS)

In [ ]:
def generate_one_hot(cluster_assignment):
    """
    Function to generate one hot encoding from the cluster assignment done using K-Means clustering.
    """
    encoded = np.zeros((cluster_assignment.size, cluster_assignment.max()+1))
    encoded[np.arange(cluster_assignment.size),cluster_assignment] = 1
    return encoded


def get_cluster_mean(cluster, cluster_probability):
    """
    Function to calculate the cluster mean parameter.
    """
    cluster_probability = cluster_probability.reshape(-1,1)
    coordinates_average = np.multiply(cluster, cluster_probability)
    cluster_mean = np.sum(coordinates_average, axis=0)/np.sum(cluster_probability)
    return cluster_mean


def get_cluster_covariance(cluster, cluster_mean, cluster_probability):
    """
    Function to calculate cluster covariance parameter
    """
    cov = np.zeros([train_coordinates.shape[1], train_coordinates.shape[1]])

    for i in range(train_coordinates.shape[0]):
        distance_metric = (train_coordinates[i]-cluster_mean).reshape(-1,1)
        cov += cluster_probability[i]*np.dot(distance_metric, distance_metric.T)
    
    return cov/np.sum(cluster_probability)


def get_mixing_weights(cluster_probability):
    """
    Function to calculate the mixing weights
    """
    return np.sum(cluster_probability)/len(cluster_probability)


def calculate_joint_gaussian(cluster, clusters):
    """
    Function to calculate the joint gaussian PDF of all the clusters
    """
    N = clusters.NUM_DATA_POINTS
    gaussian_pdf = np.zeros([N,3])
    for j in range(clusters.NUM_CLUSTERS):
        for i in range(N):
            distance_metric = (cluster[i,:] - clusters.centroids[j]).reshape(-1,1)

            # Splitting the numerator into multiple terms for readability
            _a = np.dot(distance_metric.T, np.linalg.inv(clusters.covariances[j]))
            _num = np.exp(-0.5*np.dot(_a, distance_metric))
            _denom = np.sqrt((2*np.pi)**cluster.shape[1]*np.linalg.det(clusters.covariances[j]))
            gaussian_pdf[i,j] = _num/_denom
    
    return gaussian_pdf


def compute_expectation(clusters, gaussian_pdf):
    """
    Expectation step:
    Calculate the probabilities of each point with respect to the clusters
    """
    # Denominator of expectation step is same for all clusters.
    # Compute the denominator
    _denom = 0
    for i in range(clusters.NUM_CLUSTERS):
        _denom += clusters.mixing_weights[i]*gaussian_pdf[:,i]
    
    for i in range(clusters.NUM_CLUSTERS):
        clusters.cluster_probability[:,i] = clusters.mixing_weights[i]*gaussian_pdf[:,i]/_denom
        

def compute_likelihood(mixing_weights, gaussian_pdf):
    """
    Compute the log likelihood function of the current assignment.
    """
    return np.sum((np.log(np.dot(gaussian_pdf, clusters.mixing_weights))))
    
    
        
def assign_clusters(cluster_probability):
    """
    Function to assign clusters based on the maximum probable cluster
    """
    assignment = np.argmax(cluster_probability, axis=1)
    assignment_one_hot = generate_one_hot(assignment)
    return assignment, assignment_one_hot


def plot_clusters(train_coordinates, predicted, centroid, labels_dict, ax):
    ear_left_predicted = train_coordinates[predicted == labels_dict["Ear_left"]]
    head_predicted = train_coordinates[predicted == labels_dict["Head"]]
    ear_right_predicted = train_coordinates[predicted == labels_dict["Ear_right"]]

    ax.scatter(head_predicted[:, 0], head_predicted[:, 1], color="Blue")
    ax.scatter(ear_left_predicted[:, 0], ear_left_predicted[:, 1], color="Red")
    ax.scatter(ear_right_predicted[:, 0], ear_right_predicted[:, 1], color="Green")
    ax.scatter(centroid[:, 0], centroid[:, 1], color="Black")
    ax.axis("equal")

    
def generate_confusion_matrix(actual, predicted):
    num_labels = len(np.unique(actual))
    matrix = np.zeros([num_labels, num_labels])
    
    for i in range(len(actual)):
        matrix[actual[i], predicted[i]] += 1
    
    return matrix


def plot_confusion_matrix(centroid, predicted, data, ax):
    labels_sorted = np.argsort(centroid[:,0])
    labels_original = ["Ear_left", "Head", "Ear_right"]
    labels_dict = dict(zip(labels_original, labels_sorted))
#     print(labels_dict)
    actual = np.array([labels_dict[i.split(" ")[2]] for i in data])
    
    c_matrix = generate_confusion_matrix(actual, predicted)
    
    ax.matshow(c_matrix, cmap="gray")
    ax.set_xticks(ticks=list(labels_dict.values()))
    ax.set_yticks(ticks=list(labels_dict.values()))
    ax.set_xticklabels(list(labels_dict.keys()), rotation=45)
    ax.set_yticklabels(list(labels_dict.keys()), rotation=45)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("Actual")
    for i in range(len(c_matrix)):
        for j in range(len(c_matrix)):
            ax.text(j, i, c_matrix[i,j], weight=1000, rotation=45, color="blue")
    return labels_dict


def plot_gaussians(clusters, ax):
    """
    Function to plot gaussian contours
    """
    x, y = np.mgrid[0:1:.01, 0:1:.01]
    pos = np.dstack((x, y))
    colors = ["Reds", "Blues", "Greens"]
    
    for i in range(NUMBER_OF_CLUSTERS):
        rv = multivariate_normal(clusters.centroids[i,:], clusters.covariances[i])
        pdf = rv.pdf(pos)
        ax.contour(x, y, pdf, levels=10)

In [ ]:
# Check if initial centroids information is available from the K-Means run. Else run kmeans on the data again to generate the initial clusters

if 1: #"centroid" not in locals() or "predicted" not in locals():
    print(f"Initial cluster information not available. Clustering using K-Means now!")
    centroid, predicted = kmeans2(train_coordinates, 3, minit="points")
else:
    pass

clusters.cluster_probability = generate_one_hot(predicted)

In [ ]:
# fig, ax = plt.subplots(NUMBER_OF_ITERATIONS, 3)
# fig.set_size_inches(20,20)
likelihood = []  # Define a list to store all the likelihoods
iteration_num = 0  # Initialise the number of iterations to 0

while True:
# for iteration in range(NUMBER_OF_ITERATIONS):
    # Compute the centroids and update the values
    for i in range(NUMBER_OF_CLUSTERS):
            clusters.centroids[i,:] = get_cluster_mean(train_coordinates, clusters.cluster_probability[:,i])

    # Compute covariances and update the values
    for i in range(NUMBER_OF_CLUSTERS):
            clusters.covariances[i,:,:] = get_cluster_covariance(train_coordinates, clusters.centroids[i,:], clusters.cluster_probability[:,i])

    # Compute mixing weights and update the values
    for i in range(NUMBER_OF_CLUSTERS):
            clusters.mixing_weights[i,:] = get_mixing_weights(clusters.cluster_probability[:,i])

    # Calculate Joint Gaussian PDF for the points
    gaussian_pdf = calculate_joint_gaussian(train_coordinates, clusters)

    # Expectation step
    compute_expectation(clusters, gaussian_pdf)
    
    # Compute the likelihood
    likelihood.append(compute_likelihood(clusters.mixing_weights, gaussian_pdf))
    
    # Get the assignments and plot the clusters
    if iteration_num%4 == 0 or iteration_num in range(0,4):
        fig, ax = plt.subplots(1, 3)
        fig.set_size_inches(12,4)
        assignment, _ = assign_clusters(clusters.cluster_probability)
        labels_dict = plot_confusion_matrix(clusters.centroids, assignment, data, fig.axes[0])
        plot_clusters(train_coordinates, assignment, clusters.centroids, labels_dict, fig.axes[1])
        plot_gaussians(clusters, ax=fig.axes[2])
        fig.suptitle(f'Plots at {iteration_num} iteration')
        fig.savefig(f"plots/q2_gmm_{iteration_num}_iter.png", dpi=200)
        fig.show()
    
    if likelihood[iteration_num] - likelihood[iteration_num-1] < 1e-5 and iteration_num > 1:
        break
    
    iteration_num += 1  # Increment the number of iterations

In [ ]:
plt.plot(likelihood)
plt.xlabel("Iterations")
plt.ylabel("Log Likelihood")
plt.title("Iterations vs Log Likelihood")
plt.savefig("plots/q2_likelihood.png", dpi=200)